In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()
using LinearAlgebra
using ForwardDiff
using RobotDynamics
using Ipopt
using MathOptInterface
const MOI = MathOptInterface
using Random

  Activating environment at `~/Documents/planar_quadruped_landing/Project.toml`


In [2]:
include("quadratic_cost.jl")
include("planar_quadruped.jl")
include("sparseblocks.jl")
include("utils.jl")
include("nlp.jl")
include("moi.jl")
include("costs.jl")
include("constraints.jl")
include("ref_traj.jl")

WebIO._IJuliaInit()

reference_trajectory

## Reference Trajectory

## Problem Definition

In [3]:
# Dynamics model
model = PlanarQuadruped()
g, mb, lb, l1, l2 = model.g, model.mb, model.lb, model.l1, model.l2

# Discretization
tf = 0.6
dt = 0.02
N = Int(ceil(tf / dt)) + 1
times = range(0, tf, length=N)
t_trans = 0.2

n = 14
m = 4

# Initial Conditions
# currently, we assume the initial mode ID is 1
# xinit = [-0.4;0.6;-3.1415/9;        0.0;0.0; -1.0;0.2; 0.0;-0.5;0.0; 0.0;0.0; 0.0;-0.5]
# xterm = [-lb/2;sqrt(l1^2+l2^2);0.0; 0.0;0.0; -lb;0.0;  0.0;0.0;0.0;  0.0;0.0; 0.0;0.0]
xinit = zeros(n)
xinit[1] = -lb / 2                  # xb
xinit[2] = sqrt(l1^2 + l2^2) + 0.05 # yb
xinit[3] = 10 * pi / 180          # theta
xinit[6] = -lb                    # x2
xinit[7] = 0.05                   # y2
xinit[9] = -3.0                   # yb_dot
xinit[12] = -3.0                  # y2_dot

xterm = zeros(n)
xterm[1] = -lb / 2           # xb
xterm[2] = sqrt(l1^2 + l2^2) # yb
xterm[6] = -lb             # x2

init_mode = 1

# Reference Trajectory
Xref, Uref = reference_trajectory(model, times, t_trans, xinit, xterm, init_mode)

# Objective
Random.seed!(1)
Q = Diagonal([10.0;10.0;1.0; 10.0;10.0; 10.0;10.0; 10.0;10.0;1.0; 10.0;10.0; 10.0;10.0])
R = Diagonal(fill(1e-3, 4))
Qf = Q

obj = map(1:N-1) do k
    LQRCost(Q, R, Xref[k], Uref[k])
end
push!(obj, LQRCost(Qf, R * 0, Xref[N], Uref[1]))

# Define the NLP
nlp = HybridNLP(model, obj, init_mode, tf, N, Xref[1], Xref[end]);

## Solve

In [ ]:
nlp.cinds

In [4]:
# Initial guess
xtransit = zeros(n)
xtransit[1] = -lb/2 # xb
xtransit[2] = l1    # yb
xtransit[3] = 0     # theta
xtransit[6] = -lb   # x2

# Random.seed!(1)
# Uguess = [u + 0.1*randn(length(u)) for u in Uref]
# Xguess = [x + 0.1*randn(length(x)) for x in Xref]

Uguess = [zeros(m) + rand(m) for u in Uref]
Xguess = [zeros(n) for x in Xref]

k_trans = nlp.k_trans

for i = 1:k_trans
    Xguess[i] = xinit + (xtransit - xinit)/timesteps_phase_1 * i + 0.1 * rand(n)
end

# display((xtransit - xinit)/timesteps_phase_1)

In [5]:
Z0 = packZ(nlp, Xguess, Uguess);

# Z_sol, solver = solve(Z0, nlp, c_tol=1e-6, tol=1e-6)
Z_sol, solver = solve(Z0, nlp, c_tol=1e-3, tol=1e-2)

Creating NLP Block Data...
Creating Ipopt...
Adding constraints...
starting Ipopt Solve...

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:   277000
Number of nonzeros in inequality constraint Jacobian.:    17174
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:      554
                     variables with only lower bounds:        0
                variables with lower and upper bounds:    

([-0.25003879843909993, 0.403566673669347, 0.17453207565289494, 1.2133210049382646e-6, 6.916133867779444e-5, -0.4999999890148621, 0.04999982934599231, 5.487129631459682e-6, -2.999997718102135, -2.136876383785798e-7  …  7.995709798054782e-11, -0.5000073718721654, -1373.0841346124882, 3.8836925172202095e-7, -5.780385133580343e-7, -2.6978925825533184e-9, 1.1232348379147534e-8, 1.35591677422646e-9, 1.414283308484613e-6, -4.099668109334087e-6], Ipopt.Optimizer)

In [6]:
Z_sol[1:14] - xinit

14-element Vector{Float64}:
 -3.879843909992964e-5
  1.3283076073233868e-5
 -8.49546538006507e-7
  1.2133210049382646e-6
  6.916133867779444e-5
  1.0985137910424214e-8
 -1.706540076959473e-7
  5.487129631459682e-6
  2.2818978648864174e-6
 -2.136876383785798e-7
 -3.2902049483002004e-7
  2.1785335882373857e-5
  2.553425548656151e-8
  1.267494597556015e-8

In [7]:
Z_sol[end-13:end] - xterm

14-element Vector{Float64}:
    -6.688671739307761e-6
     2.291167372769909e-6
     2.514111445635883e-7
     2.837923217051161e-9
     7.995709798054782e-11
    -7.371872165373716e-6
 -1373.0841346124882
     3.8836925172202095e-7
    -5.780385133580343e-7
    -2.6978925825533184e-9
     1.1232348379147534e-8
     1.35591677422646e-9
     1.414283308484613e-6
    -4.099668109334087e-6

EXIT: Restoration failed!

EXIT: Converged to a point of local infeasibility. Problem may be infeasible.

In [ ]:
# # display(Z_sol)
# using DelimitedFiles
# using CSV
# writedlm( "data.csv",  Z_sol, ',')